In [91]:
# notebook dependencies
import pyspark
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import regexp_extract, regexp_replace

# note: the pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, lit

# note: the following import, imports all pyspark sql functions similar to above
from pyspark.sql.functions import *

# creating the spark instance
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# pandas, numpy, and matplotlib imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# pydatasets
from pydataset import data

# tqdm loading bar library
from tqdm.notebook import tqdm, trange
import time # to be used in loop iterations

# disabling warnings
import warnings
warnings.filterwarnings('ignore')

In [92]:
# Create a spark data frame that contains your favorite programming languages.

# The name of the column should be language
# View the schema of the dataframe
# Output the shape of the dataframe
# Show the first 5 records in the dataframe

In [93]:
# creating a pyspark dataframe

df = pd.DataFrame({ 
    "n": np.arange(20), 
    "language": "python"
})

df = spark.createDataFrame(df)
df

DataFrame[n: bigint, language: string]

In [94]:
# printing the pyspark dataframe shape

print(f'pyspark df shape: {(df.count(), len(df.columns))}')

pyspark df shape: (20, 2)


In [95]:
# df schema

df.printSchema()

root
 |-- n: long (nullable = true)
 |-- language: string (nullable = true)



In [96]:
# describing the pyspark df

df.describe().show()

+-------+-----------------+--------+
|summary|                n|language|
+-------+-----------------+--------+
|  count|               20|      20|
|   mean|              9.5|    null|
| stddev|5.916079783099616|    null|
|    min|                0|  python|
|    max|               19|  python|
+-------+-----------------+--------+



In [97]:
# showing the first 5 records in the dataframe

df.show(5)

+---+--------+
|  n|language|
+---+--------+
|  0|  python|
|  1|  python|
|  2|  python|
|  3|  python|
|  4|  python|
+---+--------+
only showing top 5 rows



In [98]:
# exercise number 2: Load the mpg dataset as a spark dataframe

mpg = data("mpg")
mpg = spark.createDataFrame(mpg)

mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [99]:
# Create 1 column of output that contains a message like the one below:
# using a combination of pyspark functions (concat, expr, and lit?)

# using the "concatenate with separator method"

mpg = mpg.withColumn("concat_example", concat_ws(
    " ",
    lit("The"),
    mpg.year,
    mpg.model, 
    lit("has a"),
    mpg.cyl,
    lit("cylinder enginge."))
)

# printing the transformation/added column 
mpg.show(5) # checks out!

+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|      concat_example|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|The 1999 a4 has a...|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|The 1999 a4 has a...|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|The 2008 a4 has a...|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|The 2008 a4 has a...|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|The 1999 a4 has a...|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
only showing top 5 rows



In [100]:
#  2b. Transform the trans column so that it only contains either manual or auto.
# will use reg expression to accomplish this task

mpg = mpg.withColumn("trans", regexp_extract("trans", r'[a-z:]{4,6}', 0))

In [101]:
# printing the transformation

mpg.show(5) # checks out!

+------------+-----+-----+----+---+------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|      concat_example|
+------------+-----+-----+----+---+------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|The 1999 a4 has a...|
|        audi|   a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|The 1999 a4 has a...|
|        audi|   a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|The 2008 a4 has a...|
|        audi|   a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|The 2008 a4 has a...|
|        audi|   a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|The 1999 a4 has a...|
+------------+-----+-----+----+---+------+---+---+---+---+-------+--------------------+
only showing top 5 rows



In [102]:
# exercise number 3: Load the tips dataset as a spark dataframe.

tips_df = data("tips")
tips_df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [103]:
# creating the pyspark dataframe

tips = spark.createDataFrame(tips_df)
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [104]:
# schema print-out

tips.schema

StructType([StructField('total_bill', DoubleType(), True), StructField('tip', DoubleType(), True), StructField('sex', StringType(), True), StructField('smoker', StringType(), True), StructField('day', StringType(), True), StructField('time', StringType(), True), StructField('size', LongType(), True)])

In [118]:
# df describe 

tips.describe().show()

+-------+------------------+------------------+------+------+----+------+------------------+
|summary|        total_bill|               tip|   sex|smoker| day|  time|              size|
+-------+------------------+------------------+------+------+----+------+------------------+
|  count|               244|               244|   244|   244| 244|   244|               244|
|   mean|19.785942622950813|2.9982786885245907|  null|  null|null|  null| 2.569672131147541|
| stddev| 8.902411954856856| 1.383638189001182|  null|  null|null|  null|0.9510998047322344|
|    min|              3.07|               1.0|Female|    No| Fri|Dinner|                 1|
|    max|             50.81|              10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+------------------+------------------+------+------+----+------+------------------+



In [112]:
# df dtypes

tips.dtypes

[('total_bill', 'double'),
 ('tip', 'double'),
 ('sex', 'string'),
 ('smoker', 'string'),
 ('day', 'string'),
 ('time', 'string'),
 ('size', 'bigint')]

In [119]:
# summary stats for only numerical cols

num_cols = [item[0] for item in tips.dtypes if item[1].startswith('double') or item[1].startswith("big")]

# running the describe method on only* numerical cols
tips.describe(num_cols).show()

+-------+------------------+------------------+------------------+
|summary|        total_bill|               tip|              size|
+-------+------------------+------------------+------------------+
|  count|               244|               244|               244|
|   mean|19.785942622950813|2.9982786885245907| 2.569672131147541|
| stddev| 8.902411954856856| 1.383638189001182|0.9510998047322344|
|    min|              3.07|               1.0|                 1|
|    max|             50.81|              10.0|                 6|
+-------+------------------+------------------+------------------+



In [120]:
# creating a summary statistics df to work from

summary_stats = tips.describe(num_cols)
summary_stats.show()

+-------+------------------+------------------+------------------+
|summary|        total_bill|               tip|              size|
+-------+------------------+------------------+------------------+
|  count|               244|               244|               244|
|   mean|19.785942622950813|2.9982786885245907| 2.569672131147541|
| stddev| 8.902411954856856| 1.383638189001182|0.9510998047322344|
|    min|              3.07|               1.0|                 1|
|    max|             50.81|              10.0|                 6|
+-------+------------------+------------------+------------------+



In [106]:
# creating a "transpose" function to help manipulate the data and swap cols for rows

def TransposeDF(df, columns, pivotCol):
    '''"transpose" function to help manipulate the data and swap cols for rows'''

    columnsValue = list(map(lambda x: str("'") + str(x) + str("',")  + str(x), columns))

    stackCols = ','.join(x for x in columnsValue)

    df_1 = df.selectExpr(pivotCol, "stack(" + str(len(columns)) + "," + stackCols + ")")\
           .select(pivotCol, "col0", "col1")

    final_df = df_1.groupBy(col("col0")).pivot(pivotCol).agg(concat_ws("", collect_list(col("col1"))))\
                 .withColumnRenamed("col0", pivotCol)

    return final_df

In [121]:
# let's see if the function works

summary_stats = TransposeDF(summary_stats, num_cols, "summary")
summary_stats.show() # checks out!

+----------+-----+-----+------------------+----+------------------+
|   summary|count|  max|              mean| min|            stddev|
+----------+-----+-----+------------------+----+------------------+
|total_bill|  244|50.81|19.785942622950813|3.07| 8.902411954856856|
|       tip|  244| 10.0|2.9982786885245907| 1.0| 1.383638189001182|
|      size|  244|    6| 2.569672131147541|   1|0.9510998047322344|
+----------+-----+-----+------------------+----+------------------+



In [123]:
# creating a range column 

summary_stats = summary_stats.withColumn("range", summary_stats["max"] - summary_stats["min"])
summary_stats.show() # checks out!

+----------+-----+-----+------------------+----+------------------+-----+
|   summary|count|  max|              mean| min|            stddev|range|
+----------+-----+-----+------------------+----+------------------+-----+
|total_bill|  244|50.81|19.785942622950813|3.07| 8.902411954856856|47.74|
|       tip|  244| 10.0|2.9982786885245907| 1.0| 1.383638189001182|  9.0|
|      size|  244|    6| 2.569672131147541|   1|0.9510998047322344|  5.0|
+----------+-----+-----+------------------+----+------------------+-----+



In [124]:
# What percentage of observations are smokers?

tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [138]:
# filter for smokers in tips dataframe

smoker_df = tips.filter(tips.smoker == "Yes")
smoker_count = smoker_df.count()
smoker_count

93

In [144]:
# print the percentage of smokers in tips df

print('Percentage of smokers in tips df: {:.2f}'.format(smoker_count / tips.count()))

Percentage of smokers in tips df: 0.38
